In [3]:
driver.quit()

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

standard_sets = {
    "Dominaria unida": 5072,
    "La Guerra de los Hermanos": 5164,
    "Pirexia: Todos serán uno": 5184,
    "Marcha de las máquinas": 5227,
    "Marcha de las máquinas: las secuelas": 5320,
    "Las Tierras Salvajes de Eldraine": 5359,
    "Las cavernas perdidas de Ixalan": 5490,
    "Asesinatos en la mansión Karlov": 5561,
    "Forajidos de Cruce de Truenos": 5647,
    "Bloomburrow": 5658,
    "Duskmourn: La casa de los horrores": 5806,
    "Magic: The Gathering - Cimientos": 5852,
    "Aetherdrift": 5982,
    "Tarkir: tormenta dracónica": 6060
}

# Lista para guardar todas las cartas de todas las ediciones
all_cards = []

for set_name, expansion_id in standard_sets.items():
    url = f"https://www.cardmarket.com/es/Magic/Users/UrbanCardsMTG/Offers/Singles?idExpansion={expansion_id}"

    options = Options()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    wait = WebDriverWait(driver, 20)
    driver.get(url)

    print(f"\n\nScrapeando edición: {set_name}")
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

    # Aceptar cookies
    try:
        accept_btn = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Aceptar todas las cookies')]"))
        )
        accept_btn.click()
    except Exception as e:
        print("No se pudo aceptar cookies:", e)

    page = 1

    while True:
        print(f"Scrapeando página {page} de {set_name}...")

        try:
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.article-row"))
            )
            offers = driver.find_elements(By.CSS_SELECTOR, "div.article-row")
            print(f"Se encontraron {len(offers)} cartas.")
        except Exception as e:
            print("No se encontraron cartas:", e)
            break

        for offer in offers:
            try:
                name = offer.find_element(By.CSS_SELECTOR, "a[href*='/es/Magic/Products/Singles/']").text.strip()
            except:
                name = "N/A"

            try:
                edition = offer.find_element(By.CSS_SELECTOR, "a.expansion-symbol").get_attribute("aria-label").strip()
            except:
                edition = set_name  # fallback al nombre del set que ya conocemos

            try:
                condition = offer.find_element(By.CSS_SELECTOR, "a.article-condition span.badge").text.strip()
            except:
                try:
                    condition = offer.find_element(By.XPATH, ".//div[contains(@class, 'product-availability-details')]//span").text.strip()
                except:
                    condition = "N/A"

            try:
                language = offer.find_element(By.CSS_SELECTOR, "span.icon[aria-label]").get_attribute("aria-label").strip()
            except:
                language = "N/A"

            try:
                price = offer.find_element(By.CSS_SELECTOR, "div.price-container").text.strip().replace("€", "").replace(",", ".")
                price = float(price)
            except:
                price = -1

            try:
                quantity_element = offer.find_element(By.CSS_SELECTOR, "div.amount-container.d-none.d-md-flex.justify-content-end.me-3 span.item-count")
                quantity_text = quantity_element.text.strip()
                print(f"Cantidad encontrada: {quantity_text}")
                quantity = int(quantity_text) if quantity_text.isdigit() else -1
            except Exception as e:
                quantity = -1
                print(f"Error extrayendo cantidad: {e}")

            all_cards.append({
                "name": name,
                "edition": edition,
                "condition": condition,
                "language": language,
                "price_eur": price,
                "quantity": quantity
            })

        # Pasar a la siguiente página
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.pagination-control.btn.btn-primary.btn-sm.ms-3:not(.disabled)")
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
            time.sleep(0.5)
            next_button.click()
            page += 1

            WebDriverWait(driver, 10).until(EC.staleness_of(offers[0]))
            time.sleep(1)
        except Exception as e:
            print("No hay más páginas o error al cambiar de página:", e)
            break

    driver.quit()

# Guardar todas las cartas de todas las ediciones en un único CSV
df = pd.DataFrame(all_cards)
df.to_csv("urban_cards_estandar_inventory_all_editions.csv", index=False, encoding="utf-8-sig")
print(f"\nScraping completado. Se extrajeron {len(df)} cartas de todas las ediciones estándar.")




Scrapeando edición: Dominaria unida
Scrapeando página 1 de Dominaria unida...
Se encontraron 20 cartas.
Cantidad encontrada: 2
Cantidad encontrada: 1
Cantidad encontrada: 2
Cantidad encontrada: 4
Cantidad encontrada: 3
Cantidad encontrada: 2
Cantidad encontrada: 2
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 2
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 4
Cantidad encontrada: 4
Cantidad encontrada: 2
Cantidad encontrada: 1
Cantidad encontrada: 4
Cantidad encontrada: 1
Scrapeando página 2 de Dominaria unida...
Se encontraron 20 cartas.
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 2
Cantidad encontrada: 1
Cantidad encontrada: 2
Cantidad encontrada: 4
Cantidad encontrada: 4
Cantidad encontrada: 1
Cantidad encontrada: 1
Cantidad encontrada: 2
Cantidad encontrada: 